<a href="https://colab.research.google.com/github/osgeokr/kari-sdm/blob/main/003_environmental_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap
import geemap.colormaps as cm

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

In [7]:
import geemap
import os, requests
import pandas as pd, geopandas as gpd
import matplotlib.pyplot as plt

from google.colab import files
from google.colab import drive

In [11]:
# 대한민국 범위 좌표 지정(좌하단 : 우상단 좌표)
rectangle_coords = [
    [125.0765578311700068, 33.1124998462386984],
    [131.8727812628719960, 38.4000004985049017],
]

# 직사각형 생성
rectangle = ee.Geometry.Rectangle(rectangle_coords)

# 직사각형을 Feature로 변환
feature = ee.Feature(rectangle)

# FeatureCollection 생성
kr = ee.FeatureCollection([feature])

m = geemap.Map(layout={'height':'400px', 'width':'800px'})
m.addLayer(kr, {}, "Republic of Korea")
m.centerObject(kr, 6)
m

Map(center=[35.77459004930315, 128.47466954702105], controls=(WidgetControl(options=['position', 'transparent_…

In [13]:
# NASA SRTM Digital Elevation 30m
elevation = ee.Image("USGS/SRTMGL1_003").select('elevation').clip(kr.geometry())

# Elevation layer
Map = geemap.Map(layout={'height':'400px', 'width':'800px'})

vis_params = {'bands':['elevation'], 'min': 0, 'max': 1800, 'palette': cm.palettes.terrain}
Map.addLayer(elevation, vis_params, 'elevation')
Map.add_colorbar(vis_params, label="Elevation (m)", orientation="vertical", layer_name="elevation")
Map.centerObject(kr.geometry(), 6)
Map

Map(center=[35.77459004930315, 128.47466954702105], controls=(WidgetControl(options=['position', 'transparent_…

In [14]:
%%time

import time

# 이미지를 Google Drive에 내보내기
task = ee.batch.Export.image.toDrive(
    image=elevation,
    description='DEM',
    folder='kari-sdm',
    scale=10,  # 이미지의 해상도
    region=kr.geometry(),  # 내보낼 영역
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)

# 내보내기 작업 시작
task.start()

# 내보내기 작업 상태 확인
print('Export task started. Checking status...')

while True:
    status = task.status()
    state = status['state']
    print('Polling for task (id: {}). Status: {}'.format(task.id, state))
    if state in ['COMPLETED', 'FAILED']:
        break
    time.sleep(30)  # 30초 간격으로 상태 확인

# 완료 후 상태 출력
print('Task completed. Final status:')
print(status)

Export task started. Checking status...
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: READY
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: RUNNING
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: RUNNING
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: RUNNING
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: RUNNING
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: RUNNING
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: RUNNING
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: RUNNING
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: RUNNING
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: RUNNING
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: RUNNING
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: RUNNING
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: RUNNING
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Status: RUNNING
Polling for task (id: KZNPK7U7N3CUS46UHHCHPSKY). Sta

In [15]:
# Global Forest Cover Change (GFCC) Tree Cover Multi-Year Global 30m
tcc = (
    ee.ImageCollection("NASA/MEASURES/GFCC/TC/v3")
    .filterDate("2015-01-01", "2015-12-31")
    .select(["tree_canopy_cover"], ["TCC"])
)

# tcc에 포함된 이미지의 개수 확인
image_count = tcc.size()

# 이미지 개수 출력
print("Number of images in tcc:", image_count.getInfo())

Number of images in tcc: 8194
